# Práctica 2 - Self-Organising Maps - POKEMONS
## Preparación de entorno
#### Importar librerías de código

In [245]:
# from __future__ import division

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import patches as patches

%matplotlib inline

#### Dataset que se va a utilizar para el entrenamiento

In [267]:
# Dataset a utilizar para entrenar la red
df_pokemon = pd.read_csv('pokemon_train.csv')

# Definimos un array con los posibles tipos de los Pokemons
tipos_char = ['normal', 'dark', 'dragon', 'electric', 'fairy', 'fighter', 'fire', 'flying', 'ghost', 'grass', 'ground', 'ice', 'bug', 'poison', 'psychic', 'rock', 'steel', 'water']

# Columnas del dataset que nos importa para clasificar los tipos
col_tipos = ['type1', 'type2']

col_against = ['against_bug', 'against_dark', 'against_dragon', 'against_electric', 'against_fairy', 'against_fight', 'against_fire', 'against_flying', 'against_ghost', 'against_grass', 'against_ground', 'against_ice', 'against_normal', 'against_poison', 'against_psychic', 'against_rock', 'against_steel', 'against_water']

# Convertimos el array de tipos_char a un array numérico con igual separación entre 0 y 1 (así estaría normalizado)
tipos_num = np.linspace(0, 1, num=len(tipos_char))

# Cambiamos el type de las columnas "type1" y "type2" a string para poder comparar sus elmentos posteriormente
df_pokemon.type1 = df_pokemon.type1.astype("string")
df_pokemon.type2 = df_pokemon.type2.astype("string")

# Creamos un bucle para reemplazar el tipo de cada pokemon de su forma string a su valor correspondiente numérico (tipos_char2 --> tipos_num)
for i in range (len(df_pokemon)):
    # Si el valor de la columna "type1" se encuentra dentro de la lista "tipos_char"
    if df_pokemon["type1"][i] in tipos_char:
        # Guardamos el índice de la lista "tipos_char" para saber su valor numérico correspondiente en la lista "tipos_num"
        index1 = tipos_char.index(df_pokemon["type1"][i])
        tipos = str(tipos_num[index1])
        # Remplazamos su tipo por su corresponditen valor numérico
        df_pokemon["type1"] = df_pokemon["type1"].replace(df_pokemon["type1"][i], tipos)
    
    # Repetimos lo mismo con la columna type2
    if df_pokemon["type2"][i] in tipos_char:
        
        index2 = tipos_char.index(df_pokemon["type2"][i])
        tipos = str(tipos_num[index2])
        
        df_pokemon["type2"] = df_pokemon["type2"].replace(df_pokemon["type2"][i], tipos)
        
df_pokemon.head(15)

,id,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,against_ice,against_normal,against_poison,against_psychic,against_rock,against_steel,against_water,name,type1,type2
0,0,1.00,1.0,1.0,0.5,0.5,0.50,2.0,2.0,1.0,...,2.0,1.0,1.0,2.0,1.0,1.0,0.5,Bulbasaur,0.5294117647058824,0.7647058823529411
1,1,1.00,1.0,1.0,0.5,0.5,0.50,2.0,2.0,1.0,...,2.0,1.0,1.0,2.0,1.0,1.0,0.5,Ivysaur,0.5294117647058824,0.7647058823529411
2,2,1.00,1.0,1.0,0.5,0.5,0.50,2.0,2.0,1.0,...,2.0,1.0,1.0,2.0,1.0,1.0,0.5,Venusaur,0.5294117647058824,0.7647058823529411
3,3,0.50,1.0,1.0,1.0,0.5,1.00,0.5,1.0,1.0,...,0.5,1.0,1.0,1.0,2.0,0.5,2.0,Charmander,0.3529411764705882,-
4,4,0.50,1.0,1.0,1.0,0.5,1.00,0.5,1.0,1.0,...,0.5,1.0,1.0,1.0,2.0,0.5,2.0,Charmeleon,0.3529411764705882,-
5,5,0.25,1.0,1.0,2.0,0.5,0.50,0.5,1.0,1.0,...,1.0,1.0,1.0,1.0,4.0,0.5,2.0,Charizard,0.3529411764705882,0.4117647058823529
6,6,1.00,1.0,1.0,2.0,1.0,1.00,0.5,1.0,1.0,...,0.5,1.0,1.0,1.0,1.0,0.5,0.5,Squirtle,1.0,-
7,7,1.00,1.0,1.0,2.0,1.0,1.00,0.5,1.0,1.0,...,0.5,1.0,1.0,1.0,1.0,0.5,0.5,Wartortle,1.0,-
8,8,1.00,1.0,1.0,2.0,1.0,1.00,0.5,1.0,1.0,...,0.5,1.0,1.0,1.0,1.0,0.5,0.5,Blastoise,1.0,-
9,9,1.00,1.0,1.0,1.0,1.0,0.50,2.0,2.0,1.0,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,Caterpie,0.7058823529411764,-


### SOM Setup
#### Variables definidas por el alumno

In [268]:
# Inicializa tamaño del mapa de Kohonen, número de iteraciones y learning rate
# Inicializa normalizar_datos dependiendo de si tienes que normalizar los datos o no
lado_mapa = 30
periodo = 10000
learning_rate = 0.1
normalizar_datos = True

#### A partir de este punto solo hay cálculos. No se introducen más valores "a mano"

In [272]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.
#num_entradas = num_colores
#num_datos = valores_colores

# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario = lado_mapa / 2

# Normaliza los datos si fuese necesario dividiendo cada dato por el máximo en la matriz
if normalizar_datos:
    df_pokemon[col_against] = df_pokemon[col_against] / df_pokemon[col_against].max()
    
#df_pokemon[col_against] = df_pokemon[col_against].replace(df_pokemon[col_against], col_against_norm)    
    
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
#matriz_pesos = np.random.random((dimensiones de la matriz de pesos))
df_pokemon.head(5)

,id,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,against_ice,against_normal,against_poison,against_psychic,against_rock,against_steel,against_water,name,type1,type2
0,0,0.250,0.5,0.5,0.125,0.25,0.125,0.500,0.50,0.5,...,0.500,1.0,0.5,1.0,0.25,0.50,0.125,Bulbasaur,0.5294117647058824,0.7647058823529411
1,1,0.250,0.5,0.5,0.125,0.25,0.125,0.500,0.50,0.5,...,0.500,1.0,0.5,1.0,0.25,0.50,0.125,Ivysaur,0.5294117647058824,0.7647058823529411
2,2,0.250,0.5,0.5,0.125,0.25,0.125,0.500,0.50,0.5,...,0.500,1.0,0.5,1.0,0.25,0.50,0.125,Venusaur,0.5294117647058824,0.7647058823529411
3,3,0.125,0.5,0.5,0.250,0.25,0.250,0.125,0.25,0.5,...,0.125,1.0,0.5,0.5,0.50,0.25,0.500,Charmander,0.3529411764705882,-
4,4,0.125,0.5,0.5,0.250,0.25,0.250,0.125,0.25,0.5,...,0.125,1.0,0.5,0.5,0.50,0.25,0.500,Charmeleon,0.3529411764705882,-


#### Funciones para entrenar/clasificar

In [273]:
# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""
def calcular_bmu(patron_entrada, m_pesos, m): # patron_entrada y m_pesos, dimensiones 10 x 10
    # Dj = sqrt(N_sum(xi - wij)**2)
    bmu_index_temp = np.array([0, 0])
    dist_bmu_temp = 1000
    
    for i in range(m):     
        for j in range(m):
            temp = 0
            for k in range(num_colores):
                temp += (patron_entrada[k] - m_pesos[i][j][k])**2
            temp = np.sqrt(temp)
    
            if(temp < dist_bmu_temp):
                bmu_index_temp[0] = i
                bmu_index_temp[1] = j
                dist_bmu_temp = temp
    '''
    bmu = []
    for k in range(num_colores):
        bmu.append(m_pesos[(bmu_index_temp[0])][(bmu_index_temp[1])][k])'''
    return (bmu_index_temp)
    
    '''
    def distance(vect_a, vect_b):
            dist = np.linalg.norm(vect_a - vect_b)'''

In [274]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
def variacion_learning_rate(lr_inicial, i, n_iteraciones):
    lr_actual = lr_inicial*(1-(i/n_iteraciones))
    return lr_actual

In [275]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
    vecindario_actual = 1 + (vecindario_inicial * (1 - (i/n_iteraciones)))
    return vecindario_actual

In [276]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""

def dist_BMU(BMU_index, i, j):
    res = np.sqrt( (BMU_index[0]-i)**2  +  (BMU_index[1]-j)**2)
    return res

def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU**2 / (2*vecindario_actual**2))

#### Funciones para dibujar la salida de la red

In [278]:
# Función para pintar una matriz de valores como colores RGB
def pintar_mapa(matriz_valores, iteraciones):
    fig = plt.figure()
    
    # Establece ejes
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, matriz_pesos.shape[0]+1))
    ax.set_ylim((0, matriz_pesos.shape[1]+1))
    ax.set_title('Self-Organising Map después de %d iteraciones' % iteraciones)

    # Dibuja los rectángulos de color RGB
    for x in range(1, matriz_valores.shape[0] + 1):
        for y in range(1, matriz_valores.shape[1] + 1):
            ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1,
                         facecolor=matriz_valores[x-1,y-1,:],
                         edgecolor='none'))
    plt.show()

## SOM Entrenamiento

In [279]:
# Entrena la red con el dataset de entrenamiento




## SOM Clasificación

In [ ]:
# Clasifica los patrones de entrenamiento con la matriz de pesos recién entrenada



## SOM Prueba

In [ ]:
# Clasifica nuevos patrones

